### [01] 데이터프레임으로 만들고 출력해보기

In [ ]:
import pandas as pd

# 파일 읽어 데이터프레임으로 불러오기
df = pd.read_excel('./CAD_CAM_errors.xlsx')
print('데이터의 개수 :', len(df))

In [ ]:
df.head()

<u>**<한 걸음 더!> URL로 데이터프레임을 만들 수도 있습니다**</u>

In [ ]:
import pandas as pd

# 파일 URL 제공
# 원래 URL은 https://raw.githubusercontent.com/Mayhur23/Automating_tasks_with_python/main/Ch.8/CAD_CAM_errors.xlsx지만 너무 길어서 비틀리 서비스로 줄여두었습니다.
url = 'https://bit.ly/4eExhe6'

# url로 엑셀 파일 읽고 데이터프레임으로 불러오기
df = pd.read_excel(url)
print('데이터의 개수 : ', len(df))

### [02] 분류 요청 함수 만들기

In [ ]:
input_list = df['오류 내용 및 조치 사항'].to_list()

for input in input_list[:5]:
   print(input)

In [ ]:
import openai

client = openai.OpenAI(api_key="발급 받은 API 키")

system_prompt = '''1. 제품의 오류 내용 및 조치사항이 주어질 것입니다.
2. 제품의 오류 내용과 조치 사항을 적절히 분리하여 작성하십시오.
3. 답변은 반드시 파이썬의 딕셔너리 형태의 문자열로 작성하세요. 그 외에는 작성하지 마세요. 매우 중요합니다.
4. {"error_details" : 오류 내용을 적으세요, "corrective_actions": 조치 사항을 적으세요.}'''

user_prompt = '3D 모델 파일 손상으로 작업 중단 및 중요 레이어 누락으로 도면 오류 발생. 백업 파일 복구, 레이어 재정리 및 검토'

response = client.chat.completions.create(
   model="gpt-4o",
   messages=[
       {"role": "system", "content": system_prompt},
       {"role": "user", "content": user_prompt}
   ]
)

print(response.choices[0].message.content)

In [ ]:
import openai

client = openai.OpenAI(api_key="발급 받은 API 키")

def get_error_and_action(user_prompt):
    
   system_prompt = '''1. 제품의 오류 내용 및 조치사항이 주어질 것입니다.
   2. 제품의 오류 내용과 조치 사항을 적절히 분리하여 작성하십시오.
   3. 답변은 반드시 파이썬의 딕셔너리 형태의 문자열로 작성하세요. 그 외에는 작성하지 마세요. 매우 중요합니다.
   4. {"error_details" : 오류 내용을 적으세요, "corrective_actions": 조치 사항을 적으세요.}
   '''
   response = client.chat.completions.create(
       model="gpt-4",
       messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": user_prompt}
       ]
   )

   return response.choices[0].message.content

### [03] 분류 요청 함수 실행하고 다시 데이터프레임으로 만들기

In [ ]:
result_list = []
for input in input_list:
   result = get_error_and_action(input)
   result_list.append(result)

In [ ]:
print('5번 데이터 입력 :', input_list[5])
print('5번 데이터 처리 결과 :', result_list[5])

In [ ]:
print('데이터의 타입 :', type(result_list[5]))

In [ ]:
import ast

dict_sample = ast.literal_eval(result_list[5])
print('데이터의 타입 :', type(dict_sample))

In [ ]:
#  result_list의 모든 요소를 전부 딕셔너리로 변경
dict_result_list = []

for str_result in result_list:
   dict_result = ast.literal_eval(str_result)
   dict_result_list.append(dict_result)

In [ ]:
df['오류 내용'] = error_details
df['조치 사항'] = corrective_actions
df[['날짜', '고객사', '오류 내용', '조치 사항']].head()